<a href="https://colab.research.google.com/github/averma12/Deepseek_experiments/blob/main/R1_1_5bit_quantized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import subprocess
import os
from IPython.display import HTML, display
import time

# Install dependencies
def run_command(cmd):
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    return output.decode(), error.decode()

# Setup steps
setup_commands = [
    "git clone https://github.com/ggerganov/llama.cpp",
    "cmake llama.cpp -B llama.cpp/build -DBUILD_SHARED_LIBS=OFF -DGGML_CUDA=ON -DLLAMA_CURL=ON",
    "cmake --build llama.cpp/build --config Release -j"
]

print("Setting up llama.cpp...")
for cmd in setup_commands:
    out, err = run_command(cmd)
    if err:
        print(f"Error: {err}")

# Download model
print("Downloading model...")
from huggingface_hub import snapshot_download

model_path = snapshot_download(
    repo_id="unsloth/DeepSeek-R1-GGUF",
    local_dir="/content/drive/MyDrive/llama_models",  # Changed to Drive path
    allow_patterns=["*UD-IQ1_S*"]
)

# GPU Monitoring function
def monitor_gpu():
    while True:
        out, _ = run_command("nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total --format=csv -l 1")
        print(out)
        time.sleep(5)


Setting up llama.cpp...
Error: fatal: destination path 'llama.cpp' already exists and is not an empty directory.

Error: -- CUDA host compiler is GNU 11.4.0




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf:   0%|          | 0.00/41.5G [00:00<?, ?B/s]

DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf:   0%|          | 0.00/49.3G [00:00<?, ?B/s]

DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf:   0%|          | 0.00/49.4G [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!./llama.cpp/build/bin/llama-gguf-split --merge \
    /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf \
    /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf


gguf_merge: /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf -> /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf
gguf_merge: reading metadata /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf ...done
gguf_merge: reading metadata /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf ...done
gguf_merge: reading metadata /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf ...done
gguf_merge: writing tensors /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf ...done
gguf_merge: writing tensors /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf ...done
gguf_merge: writing tensors /content/drive/MyDrive/llama_models/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf ...done
gguf_merge: /content/

In [9]:
!ls -la /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf

-rw------- 1 root root 140231438240 Jan 29 08:14 /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf


In [11]:
!ls -la /content

total 24
drwxr-xr-x  1 root root 4096 Jan 29 06:30 .
drwxr-xr-x  1 root root 4096 Jan 29 06:05 ..
drwxr-xr-x  4 root root 4096 Jan  6 14:19 .config
drwx------  6 root root 4096 Jan 29 06:29 drive
drwxr-xr-x 25 root root 4096 Jan 29 06:29 llama.cpp
drwxr-xr-x  1 root root 4096 Jan  6 14:19 sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Create a directory for the model in Google Drive
!mkdir -p /content/drive/MyDrive/llama_models


Mounted at /content/drive


In [4]:
!du -h --max-depth=1 /content | sort -rh

^C


In [3]:
!mv model/DeepSeek-R1-UD-IQ1_S/*.gguf /content/drive/MyDrive/llama_models/

^C


In [3]:
!ls model/DeepSeek-R1-UD-IQ1_S/

DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf  DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf
DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf


In [4]:
!./llama.cpp/build/bin/llama-gguf-split --merge \
    model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf \
    /content/drive/MyDrive/llama_models/deepseek_merged.gguf

gguf_merge: model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf -> /content/drive/MyDrive/llama_models/deepseek_merged.gguf
gguf_merge: reading metadata model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf ...done
gguf_merge: reading metadata model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf ...done
gguf_merge: reading metadata model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf ...done
gguf_merge: writing tensors model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf ...done
gguf_merge: writing tensors model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf ...^C


In [44]:
import select
def setup_inference_config():
    base_cmd = "llama.cpp/build/bin/llama-cli"
    model_path = "/content/drive/MyDrive/llama_models/deepseek_merged-2.gguf"

    config = {
        "model": model_path,
        "n-gpu-layers": "61",
        "threads": "8",
        "ctx-size": "4096",
        "batch-size": "64",
        "temp": "0.6",
        "cache-type-k": "q4_0",
        "chat-template": "deepseek3",  # Updated to latest DeepSeek template
        "interactive": "",            # Added interactive mode
        "no-mmap": "",
        "verbose": ""
    }

    cmd = f"{base_cmd} " + " ".join([f"--{k} {v}" for k, v in config.items() if v])
    return cmd







# Check paths
import os
print("Current directory:", os.getcwd())
print("llama-cli exists:", os.path.exists("llama.cpp/build/bin/llama-cli"))
print("Model exists:", os.path.exists("/content/drive/MyDrive/llama_models/deepseek_merged-2.gguf"))


Current directory: /content
llama-cli exists: True
Model exists: True


In [ ]:
import threading

def monitor_gpu_thread():
    while True:
        subprocess.run(['nvidia-smi'], shell=True)
        time.sleep(1)

# Start GPU monitoring in background
monitor_thread = threading.Thread(target=monitor_gpu_thread, daemon=True)
monitor_thread.start()

In [37]:
!cmake llama.cpp -B llama.cpp/build \
    -DGGML_CUDA=ON \
    -DCMAKE_CUDA_ARCHITECTURES=80 \
    -DGGML_CUDA_FORCE_DMMV=ON \
    -DGGML_CUDA_FORCE_MMV=ON

!cmake --build llama.cpp/build --config Release -j


-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- CUDA Toolkit found
-- Using CUDA architectures: 80
-- CUDA host compiler is GNU 11.4.0

-- Including CUDA backend
-- Configuring done (0.5s)
-- Generating done (0.3s)
-- Build files have been written to: /content/llama.cpp/build
[  0%] Built target build_info
[  1%] Built target xxhash
[  1%] Built target sha256
[  1%] Built target sha1
[  4%] Built target ggml-base
[  6%] Built target ggml-cpu
[ 49%] Built target ggml-cuda
[ 49%] Built target ggml
[ 50%] Built target llama-gguf-hash
[ 50%] Built target llama-gguf
[ 55%] Built target llama
[ 55%] Built target test-c
[ 56%] Built target llama-simple
[ 57%] Built target llama-simple-chat
[ 58%] Built target llama-quantize-stats
[ 59%] Built target llava
[ 60%] Built target llava_static
[

In [23]:
!rm -rf llama.cpp/build
!cmake llama.cpp -B llama.cpp/build \
    -DGGML_CUDA=ON \
    -DCMAKE_CUDA_ARCHITECTURES=80 \
    -DGGML_CUDA_F16=ON \
    -DGGML_CUDA_FORCE_CUBLAS=ON \
    -DGGML_CUDA_FA_ALL_QUANTS=ON

!cmake --build llama.cpp/build --config Release -j


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5

In [26]:
!ldd llama.cpp/build/bin/llama-cli | grep cuda

	libggml-cuda.so => /content/llama.cpp/build/bin/libggml-cuda.so (0x00007e3643d92000)
	libcudart.so.12 => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.12 (0x00007e3643a00000)
	libcublas.so.12 => /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.12 (0x00007e363d200000)
	libcuda.so.1 => /usr/lib64-nvidia/libcuda.so.1 (0x00007e363b400000)
	libcublasLt.so.12 => /usr/local/cuda/targets/x86_64-linux/lib/libcublasLt.so.12 (0x00007e3618400000)


In [25]:
def run_inference():
    cmd = setup_inference_config()
    print("\n=== Starting with minimal config ===")
    print("Command:", cmd)

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print("\nOutput:", result.stdout)
    print("\nErrors:", result.stderr)
    print("\nReturn code:", result.returncode)
run_inference()


=== Starting with minimal config ===
Command: llama.cpp/build/bin/llama-cli --model /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf --n-gpu-layers 61 --threads 8 --ctx-size 4096 --batch-size 64 --temp 0.6 --cache-type-k q4_0 --gpu-memory-utilization 0.9

Output: 

Errors: ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: yes
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
error: invalid argument: --gpu-memory-utilization


Return code: 1


In [33]:
def run_inference():
    cmd = setup_inference_config()
    print("\n=== Starting with configured settings ===")
    print("Command:", cmd)

    try:
        process = subprocess.Popen(
            cmd,
            shell=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # Wait a bit and check if process is still alive
        time.sleep(5)
        if process.poll() is not None:
            print(f"Process failed to start with error code: {process.poll()}")
            error_output = process.stderr.read()
            print(f"Error output: {error_output}")
            return None

        return process

    except Exception as e:
        print(f"Startup Error: {str(e)}")
        return None

def chat_with_model(process):
    def send_prompt(prompt):
        try:
            print("\nSending prompt...")
            process.stdin.write(prompt + '\n')
            process.stdin.flush()
            print("Prompt sent.")
        except Exception as e:
            print(f"Error sending prompt: {str(e)}")

    def read_response():
        print("\nReading response...")
        output = []
        start_time = time.time()
        no_data_count = 0

        while True:
            if process.poll() is not None:
                print(f"\nProcess ended with code: {process.returncode}")
                error_output = process.stderr.read()
                if error_output:
                    print(f"Error output: {error_output}")
                return None

            ready = select.select([process.stdout], [], [], 5.0)[0]
            if ready:
                line = process.stdout.readline()
                if line:
                    print(".", end="", flush=True)
                    output.append(line)
                    no_data_count = 0

                    if '<｜User｜>' in line:
                        print("\nResponse complete")
                        break
            else:
                no_data_count += 1
                if no_data_count > 30:  # 60 seconds timeout
                    print("\nResponse timeout")
                    break

        return ''.join(output) if output else "No response received"

    try:
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() in ['exit', 'quit']:
                break

            formatted_prompt = f"<｜User｜>{user_input}<｜Assistant｜>"
            send_prompt(formatted_prompt)
            response = read_response()

            if response is None:
                print("Model process ended, exiting...")
                break

            print("\nAssistant:", response)

    except KeyboardInterrupt:
        print("\nExiting chat...")
    finally:
        process.terminate()


In [34]:
def check_gpu_usage():
    !nvidia-smi
check_gpu_usage()

Wed Jan 29 09:30:20 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [38]:
! llama.cpp/build/bin/llama-cli --version

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: yes
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
version: 4580 (325afb37)
built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu


In [39]:
! llama.cpp/build/bin/llama-cli --help

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: yes
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
----- common params -----

-h,    --help, --usage                  print usage and exit
--version                               show version and build info
--verbose-prompt                        print a verbose prompt before generation (default: false)
-t,    --threads N                      number of threads to use during generation (default: -1)
                                        (env: LLAMA_ARG_THREADS)
-tb,   --threads-batch N                number of threads to use during batch and prompt processing (default:
                                        same as --threads)
-C,    --cpu-mask M                     CPU affinity mask: arbitrarily long hex. Complements cpu-range
                                        (default: "")
-Cr,   --cpu-range lo-hi                range of CPUs for affinit

In [47]:
!ps aux | grep llama-cli

root       67579  0.0  0.0   6484  2228 ?        S    09:52   0:00 grep llama-cli


In [48]:
def verify_model():
    model_path = "/content/drive/MyDrive/llama_models/deepseek_merged-2.gguf"
    try:
        size = os.path.getsize(model_path) / (1024**3)
        print(f"Model found, size: {size:.2f} GB")
        return True
    except FileNotFoundError:
        print("Model file not found!")
        return False

def check_gpu_usage():
    try:
        gpu_info = !nvidia-smi
        print("\nGPU Status:")
        print("\n".join(gpu_info))
    except:
        print("Failed to get GPU info")

def run_inference():
    if not verify_model():
        return None

    cmd = setup_inference_config()
    print("\n=== Starting with configured settings ===")
    print("Command:", cmd)
    print("\nInitial GPU state:")
    check_gpu_usage()

    try:
        process = subprocess.Popen(
            cmd,
            shell=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # Wait and monitor startup
        print("\nWaiting for model to load...")
        time.sleep(10)  # Increased wait time for model loading

        if process.poll() is not None:
            print(f"Process failed to start with error code: {process.poll()}")
            error_output = process.stderr.read()
            print(f"Error output: {error_output}")
            return None

        print("\nGPU state after model load:")
        check_gpu_usage()
        return process

    except Exception as e:
        print(f"Startup Error: {str(e)}")
        return None

def chat_with_model(process):
    def send_prompt(prompt):
        try:
            print("\nSending prompt...")
            process.stdin.write(prompt + '\n')
            process.stdin.flush()
            print("Prompt sent.")
            check_gpu_usage()  # Monitor GPU after sending prompt
        except Exception as e:
            print(f"Error sending prompt: {str(e)}")

    def read_response():
        print("\nReading response...")
        output = []
        start_time = time.time()
        no_data_count = 0

        while True:
            if process.poll() is not None:
                print(f"\nProcess ended with code: {process.returncode}")
                error_output = process.stderr.read()
                if error_output:
                    print(f"Error output: {error_output}")
                return None

            ready = select.select([process.stdout], [], [], 5.0)[0]
            if ready:
                line = process.stdout.readline()
                if line:
                    print(".", end="", flush=True)
                    output.append(line)
                    no_data_count = 0

                    if '<｜User｜>' in line:
                        print("\nResponse complete")
                        check_gpu_usage()  # Monitor GPU after response
                        break
            else:
                no_data_count += 1
                if no_data_count > 12:
                    print("\nResponse timeout")
                    check_gpu_usage()  # Check GPU on timeout
                    break

        return ''.join(output) if output else "No response received"

    try:
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() in ['exit', 'quit']:
                break

            send_prompt(user_input)
            response = read_response()

            if response is None:
                print("Model process ended, exiting...")
                break

            print("\nAssistant:", response)
            check_gpu_usage()  # Monitor GPU after each complete interaction

    except KeyboardInterrupt:
        print("\nExiting chat...")
    finally:
        process.terminate()
        print("\nFinal GPU state:")
        check_gpu_usage()


In [49]:
def run_inference():
    if not verify_model():
        return None

    cmd = setup_inference_config()
    print("\n=== Starting with configured settings ===")
    print("Command:", cmd)

    try:
        process = subprocess.Popen(
            cmd,
            shell=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # Immediately check for any startup errors
        time.sleep(5)
        if process.poll() is not None:
            print("Process terminated immediately!")
            error = process.stderr.read()
            print("Error output:", error)
            return None

        return process

    except Exception as e:
        print(f"Startup Error: {str(e)}")
        return None


In [50]:
# 1. Start the inference process
process = run_inference()

# 2. If process started successfully, begin chat
if process:
    print("\nModel loaded! You can start chatting.")
    print("Example: Ask a question and press Enter")
    print("Use Ctrl+C to exit")
    chat_with_model(process)
else:
    print("Failed to start model")


Model found, size: 130.60 GB

=== Starting with configured settings ===
Command: llama.cpp/build/bin/llama-cli --model /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf --n-gpu-layers 61 --threads 8 --ctx-size 4096 --batch-size 64 --temp 0.6 --cache-type-k q4_0 --chat-template deepseek3

Model loaded! You can start chatting.
Example: Ask a question and press Enter
Use Ctrl+C to exit

You: Hello again

Sending prompt...
Prompt sent.

GPU Status:
Wed Jan 29 09:54:41 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                 

In [55]:
!./llama.cpp/build/bin/llama-cli \
    --model /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf \
    --cache-type-k q4_0 \
    --threads 12 \
    --no-mmap \
    --n-gpu-layers 14 \
    --interactive \
    --prio 2 \
    --temp 0.6 \
    --ctx-size 8192 \
    --prompt "<｜User｜>What is flappy bird.<｜Assistant｜>"


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: yes
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
build: 4580 (325afb37) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA A100-SXM4-40GB) - 40090 MiB free
llama_model_loader: loaded meta data with 52 key-value pairs and 1025 tensors from /content/drive/MyDrive/llama_models/deepseek_merged-2.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = deepseek2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str    

In [ ]:
!cat llama.cpp/build/CMakeCache.txt | grep GGML_CUDA

GGML_CUDA:BOOL=ON
GGML_CUDA_F16:BOOL=ON
GGML_CUDA_FA_ALL_QUANTS:BOOL=ON
GGML_CUDA_FORCE_CUBLAS:BOOL=ON
GGML_CUDA_FORCE_MMQ:BOOL=OFF
GGML_CUDA_GRAPHS:BOOL=ON
GGML_CUDA_NO_PEER_COPY:BOOL=OFF
GGML_CUDA_NO_VMM:BOOL=OFF
GGML_CUDA_PEER_MAX_BATCH_SIZE:STRING=128


In [ ]:
cmd = setup_inference_config()

In [ ]:
run_inference()

Starting DeepSeek R1...
Command: ./llama.cpp/build/bin/llama-cli --model model/DeepSeek-R1-UD-IQ1_S/merged_file.gguf --cache-type-k q4_0 --threads 8 --n-gpu-layers 15 --prio 2 --ctx-size 4096 --temp 0.6

Model loaded! You can start chatting (Ctrl+C to exit)
Example prompt: '<｜User｜>What is calculus?<｜Assistant｜>'


<Popen: returncode: None args: './llama.cpp/build/bin/llama-cli --model mode...>

In [ ]:
def chat_with_model(process):
    def send_prompt(prompt):
        process.stdin.write(prompt + '\n')
        process.stdin.flush()

    def read_response():
        output = []
        while True:
            line = process.stdout.readline()
            if not line or '<｜User｜>' in line:
                break
            output.append(line)
        return ''.join(output)

    try:
        while True:
            user_input = input("You: ")
            formatted_prompt = f"<｜User｜>{user_input}<｜Assistant｜>"
            send_prompt(formatted_prompt)
            response = read_response()
            print("\nAssistant:", response)

    except KeyboardInterrupt:
        print("\nExiting chat...")
        process.terminate()


In [ ]:
import glob
print("Available files:")
files = glob.glob("model/DeepSeek-R1-UD-IQ1_S/*")
for f in files:
    print(f)


Available files:
model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00002-of-00003.gguf
model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf
model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00003-of-00003.gguf


In [ ]:
def merge_model_files():
    print("Merging model files...")
    input_files = "model/DeepSeek-R1-UD-IQ1_S/DeepSeek-R1-UD-IQ1_S-00001-of-00003.gguf"
    merge_cmd = f"./llama.cpp/build/bin/llama-gguf-split --merge {input_files} merged_model.gguf"

    try:
        result = subprocess.run(merge_cmd, shell=True, capture_output=True, text=True)
        print("Command output:", result.stdout)
        print("Command error:", result.stderr)
        if result.returncode == 0:
            print("Model files merged successfully")
            return True
        else:
            print(f"Error merging files: {result.stderr}")
            return False
    except Exception as e:
        print(f"Error during merge: {e}")
        return False

merge_model_files()

Merging model files...


In [ ]:
# 1. First run the process
process = run_inference()
print(process)
if not process:
    raise Exception("Error")
# 2. Then start the chat loop
chat_with_model(process)

Starting DeepSeek R1...
Command: ./llama.cpp/build/bin/llama-cli --model model/DeepSeek-R1-UD-IQ1_S/merged_file.gguf --cache-type-k q4_0 --threads 8 --n-gpu-layers 15 --prio 2 --ctx-size 4096 --temp 0.6
Error: Model file not found. Need to merge model files first.
None


Exception: Error